In [ ]:
# default_exp util

# Util

> API details.

In [ ]:
#export
from numpy.fft import fftfreq
import numpy as np
from smpr3d.torch_imports import *

In [ ]:
#export
def fftshift_checkerboard(w, h):
    re = np.r_[w * [-1, 1]]  # even-numbered rows
    ro = np.r_[w * [1, -1]]  # odd-numbered rows
    return np.row_stack(h * (re, ro))

In [ ]:
#export
def cartesian_aberrations(qx, qy, lam, C):
    """
    Zernike polynomials in the cartesian coordinate system
    :param qx:
    :param qy:
    :param lam: wavelength in Angstrom
    :param C:   12 x D
    :return:
    """

    u = qx * lam
    v = qy * lam
    u2 = u ** 2
    u3 = u ** 3
    u4 = u ** 4
    # u5 = u ** 5

    v2 = v ** 2
    v3 = v ** 3
    v4 = v ** 4
    # v5 = v ** 5

    aberr = Param()
    aberr.C1 = C[0].unsqueeze(1).unsqueeze(1)
    aberr.C12a = C[1].unsqueeze(1).unsqueeze(1)
    aberr.C12b = C[2].unsqueeze(1).unsqueeze(1)
    aberr.C21a = C[3].unsqueeze(1).unsqueeze(1)
    aberr.C21b = C[4].unsqueeze(1).unsqueeze(1)
    aberr.C23a = C[5].unsqueeze(1).unsqueeze(1)
    aberr.C23b = C[6].unsqueeze(1).unsqueeze(1)
    aberr.C3 = C[7].unsqueeze(1).unsqueeze(1)
    aberr.C32a = C[8].unsqueeze(1).unsqueeze(1)
    aberr.C32b = C[9].unsqueeze(1).unsqueeze(1)
    aberr.C34a = C[10].unsqueeze(1).unsqueeze(1)
    aberr.C34b = C[11].unsqueeze(1).unsqueeze(1)

    chi = 0

    # r-2 = x-2 +y-2.
    chi += 1 / 2 * aberr.C1 * (u2 + v2) # r^2
    #r-2 cos(2*phi) = x"2 -y-2.
    # r-2 sin(2*phi) = 2*x*y.
    chi += 1 / 2 * (aberr.C12a * (u2 - v2) + 2 * aberr.C12b * u * v) # r^2 cos(2 phi) + r^2 sin(2 phi)
    # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
    chi += 1 / 3 * (aberr.C23a * (u3 - 3 * u * v2) + aberr.C23b * (3 * u2 * v - v3))# r^3 cos(3phi) + r^3 sin(3 phi)
    # r-3 cos(phi) = x-3 +x*y-2.
    # r-3 sin(phi) = y*x-2 +y-3.
    chi += 1 / 3 * (aberr.C21a * (u3 + u * v2) + aberr.C21b * (v3 + u2 * v))# r^3 cos(phi) + r^3 sin(phi)
    # r-4 = x-4 +2*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C3 * (u4 + v4 + 2 * u2 * v2)# r^4
    # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C34a * (u4 - 6 * u2 * v2 + v4)# r^4 cos(4 phi)
    # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
    chi += 1 / 4 * aberr.C34b * (4 * u3 * v - 4 * u * v3) # r^4 sin(4 phi)
    # r-4 cos(2*phi) = x-4 -y-4.
    chi += 1 / 4 * aberr.C32a * (u4 - v4)
    # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
    chi += 1 / 4 * aberr.C32b * (2 * u3 * v + 2 * u * v3)
    # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
    # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
    # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
    # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
    # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
    # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

    chi *= 2 * np.pi / lam

    return chi

In [ ]:
#export
def memory_mb(x, dtype=None):
    if isinstance(x, th.Tensor):
        return x.nelement() * x.element_size() / 2 ** 20
    elif isinstance(x, tuple):
        assert dtype is not None, 'memory_mb: dtype must not be None'
        element_size = th.zeros(1, dtype=dtype).element_size()
        nelement = np.prod(np.asarray(x))
        return nelement * element_size / 2 ** 20


def memory_gb(x, dtype=None):
    if isinstance(x, th.Tensor):
        return x.nelement() * x.element_size() / 2 ** 30
    elif isinstance(x, tuple):
        assert dtype is not None, 'memory_mb: dtype must not be None'
        element_size = th.zeros(1, dtype=dtype).element_size()
        nelement = np.prod(np.asarray(x))
        return nelement * element_size / 2 ** 30

In [ ]:
#export
def fourier_coordinates_2D(N, dx=[1.0, 1.0], centered=True):
    qxx = fftfreq(N[1], dx[1])
    qyy = fftfreq(N[0], dx[0])
    if centered:
        qxx += 0.5 / N[1] / dx[1]
        qyy += 0.5 / N[0] / dx[0]
    qx, qy = np.meshgrid(qxx, qyy)
    q = np.array([qy, qx]).astype(np.float32)
    return q

In [ ]:
#export
def array_split_divpoints(ary, indices_or_sections, axis=0):
    """
    Split an array into multiple sub-arrays.
    Please refer to the ``split`` documentation.  The only difference
    between these functions is that ``array_split`` allows
    `indices_or_sections` to be an integer that does *not* equally
    divide the axis. For an array of length l that should be split
    into n sections, it returns l % n sub-arrays of size l//n + 1
    and the rest of size l//n.
    See Also
    --------
    split : Split array into multiple sub-arrays of equal size.
    Examples
    --------
    >>> x = np.arange(8.0)
    >>> np.array_split(x, 3)
        [array([0.,  1.,  2.]), array([3.,  4.,  5.]), array([6.,  7.])]
    >>> x = np.arange(7.0)
    >>> np.array_split(x, 3)
        [array([0.,  1.,  2.]), array([3.,  4.]), array([5.,  6.])]
    """
    try:
        Ntotal = ary.shape[axis]
    except AttributeError:
        Ntotal = len(ary)
    try:
        # handle array case.
        Nsections = len(indices_or_sections) + 1
        div_points = [0] + list(indices_or_sections) + [Ntotal]
    except TypeError:
        # indices_or_sections is a scalar, not an array.
        Nsections = int(indices_or_sections)
        if Nsections <= 0:
            raise ValueError('number sections must be larger than 0.')
        Neach_section, extras = divmod(Ntotal, Nsections)
        section_sizes = ([0] +
                         extras * [Neach_section + 1] +
                         (Nsections - extras) * [Neach_section])
        div_points = np.array(section_sizes, dtype=np.intp).cumsum()

    return div_points

In [ ]:
#export
def R_factor(z, a, world_size=1):
    """Calculate R error metric = sum(|z-a|)/sum(|a|)."""
    num = th.norm(th.abs(z) - a,p=1)
    denom = th.norm(a,p=1)
    if world_size > 1:
        dist.all_reduce(num)
        dist.all_reduce(denom)
    return num / denom

In [ ]:
#export
def distance(z, x):
    """
    Distance of two complex vectors
    :param z: tensor
    :param x: tensor
    :return:
    """
    c = th.vdot(z.ravel(), x.ravel())
    phi = -th.angle(c)
    exp_minus_phi = th.exp(1j * phi)
    p = exp_minus_phi.to(x.device)
    x_hat = x * p
    res = th.norm(z - x_hat,2)
    return res

In [ ]:
#export
def rel_dist(z, x):
    """
    Distance of two complex vectors
    :param z: tensor
    :param x: tensor
    :return:
    """
    d = distance(z, x)
    x_norm = th.norm(x,2)
    return d / x_norm

In [ ]:
#export
PARAM_PREFIX = 'pars'
class Param(dict):
    """
    Convenience class: a dictionary that gives access to its keys
    through attributes.
    
    Note: dictionaries stored in this class are also automatically converted
    to Param objects:
    >>> p = Param()
    >>> p.x = {}
    >>> p
    Param({})
    
    While dict(p) returns a dictionary, it is not recursive, so it is better in this case
    to use p.todict(). However, p.todict does not check for infinite recursion. So please
    don't store a dictionary (or a Param) inside itself.
    
    BE: Please note also that the recursive behavior of the update function will create
    new references. This will lead inconsistency if other objects refer to dicts or Params
    in the updated Param instance. 
    """
    _display_items_as_attributes = True
    _PREFIX = PARAM_PREFIX

    def __init__(self, __d__=None, **kwargs):
        """
        A Dictionary that enables access to its keys as attributes.
        Same constructor as dict.
        """
        dict.__init__(self)
        if __d__ is not None: self.update(__d__)
        self.update(kwargs)

    def __getstate__(self):
        return self.__dict__.items()

    def __setstate__(self, items):
        for key, val in items:
            self.__dict__[key] = val

    def __repr__(self):
        return "%s(%s)" % (self.__class__.__name__, dict.__repr__(self))

    # def __str__(self):
    #     from .verbose import report
    #     return report(self,depth=7,noheader=True)

    def __setitem__(self, key, value):
        # BE: original behavior modified as implicit conversion may destroy references
        # Use update(value,Convert=True) instead
        # return super(Param, self).__setitem__(key, Param(value) if type(value) == dict else value)
        return super(Param, self).__setitem__(key, value)

    def __getitem__(self, name):
        # item = super(Param, self).__getitem__(name)
        # return Param(item) if type(item) == dict else item
        return super(Param, self).__getitem__(name)

    def __delitem__(self, name):
        return super(Param, self).__delitem__(name)

    def __delattr__(self, name):
        return super(Param, self).__delitem__(name)

    # __getattr__ = __getitem__
    def __getattr__(self, name):
        try:
            return self.__getitem__(name)
        except KeyError as ke:
            raise AttributeError(ke)

    __setattr__ = __setitem__

    def copy(self, depth=0):
        """
        :returns Param: A (recursive) copy of P with depth `depth` 
        """
        d = Param(self)
        if depth > 0:
            for k, v in d.iteritems():
                if isinstance(v, self.__class__): d[k] = v.copy(depth - 1)
        return d

    def __dir__(self):
        """
        Defined to include the keys when using dir(). Useful for
        tab completion in e.g. ipython.
        If you do not wish the dict key's be displayed as attributes
        (although they are still accessible as such) set the class 
        attribute `_display_items_as_attributes` to False. Default is
        True.
        """
        if self._display_items_as_attributes:
            return self.keys()
            # return [item.__dict__.get('name',str(key)) for key,item in self.iteritems()]
        else:
            return []

    def update(self, __d__=None, in_place_depth=0, Convert=False, **kwargs):
        """
        Update Param - almost same behavior as dict.update, except
        that all dictionaries are converted to Param if `Convert` is set 
        to True, and update may occur in-place recursively for other Param
        instances that self refers to.
        
        Parameters
        ----------
        Convert : bool 
                  If True, convert all dict-like values in self also to Param.
                  *WARNING* 
                  This mey result in misdirected references in your environment
        in_place_depth : int 
                  Counter for recursive in-place updates 
                  If the counter reaches zero, the Param to a key is
                  replaced instead of updated
        """

        def _k_v_update(k, v):
            # If an element is itself a dict, convert it to Param
            if Convert and hasattr(v, 'keys'):
                # print 'converting'
                v = Param(v)
            # new key 
            if not k in self:
                self[k] = v
            # If this key already exists and is already dict-like, update it
            elif in_place_depth > 0 and hasattr(v, 'keys') and isinstance(self[k], self.__class__):
                self[k].update(v, in_place_depth - 1)
                """
                if isinstance(self[k],self.__class__):
                    # Param gets recursive in_place updates
                    self[k].update(v, in_place_depth - 1)
                else:
                    # dicts are only updated in-place once
                    self[k].update(v)
                """
            # Otherwise just replace it
            else:
                self[k] = v

        if __d__ is not None:
            if hasattr(__d__, 'keys'):
                # Iterate through dict-like argument
                for k, v in __d__.items():
                    _k_v_update(k, v)

            else:
                # here we assume a (key,value) list.
                for (k, v) in __d__:
                    _k_v_update(k, v)

        for k, v in kwargs.items():
            _k_v_update(k, v)

        return None

    def _to_dict(self, Recursive=False):
        """
        Convert to dictionary (recursively if needed).
        """
        if not Recursive:
            return dict(self)
        else:
            d = dict(self)
            for k, v in d.items():
                if isinstance(v, self.__class__): d[k] = v._to_dict(Recursive)
        return d

    @classmethod
    def _from_dict(cls, dct):
        """
        Make Param from dict. This is similar to the __init__ call
        """
        # p=Param()
        # p.update(dct.copy())
        return Param(dct.copy())


def validate_standard_param(sp, p=None, prefix=None):
    """\
    validate_standard_param(sp) checks if sp follows the standard parameter convention.
    validate_standard_param(sp, p) attemps to check if p is a valid implementation of sp.

    NOT VERY SOPHISTICATED FOR NOW!
    """
    if p is None:
        good = True
        for k, v in sp.iteritems():
            if k.startswith('_'): continue
            if type(v) == type(sp):
                pref = k if prefix is None else '.'.join([prefix, k])
                good &= validate_standard_param(v, prefix=pref)
                continue
            else:
                try:
                    a, b, c = v
                    if prefix is not None:
                        print('    %s.%s = %s' % (prefix, k, str(v)))
                    else:
                        print('    %s = %s' % (k, str(v)))
                except:
                    good = False
                    if prefix is not None:
                        print('!!! %s.%s = %s <--- Incorrect' % (prefix, k, str(v)))
                    else:
                        print('!!! %s = %s <--- Incorrect' % (k, str(v)))

        return good
    else:
        raise RuntimeError('Checking if a param fits with a standard is not yet implemented')


def format_standard_param(p):
    """\
    Pretty-print a Standard Param class.
    """
    lines = []
    if not validate_standard_param(p):
        print('Standard parameter does not')
    for k, v in p.iteritems():
        if k.startswith('_'): continue
        if type(v) == type(p):
            sublines = format_standard_param(v)
            lines += [k + '.' + s for s in sublines]
        else:
            lines += ['%s = %s #[%s] %s' % (k, str(v[1]), v[0], v[2])]
    return lines


def asParam(obj):
    """
    Convert the input to a Param.
    
    Parameters
    ----------
    a : dict_like
        Input structure, in any format that can be converted to a Param.
        
    Returns:
    out : Param
        The Param structure built from a. No copy is done if the input
        is already a Param.  
    """
    return obj if isinstance(obj, Param) else Param(obj)


def make_default(default_dict_or_file):
    """
    convert description dict to a module dict using a possibly verbose Q & A game
    """
    pass

In [3]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_util.ipynb.
Converted 10a_fasta.ipynb.
